In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('./code')
import get_mta_data as gmd
import process_mta_data as pmd

In [2]:
mta = gmd.load_local_data([2018], [4, 5])

In [ ]:
mta

In [ ]:
mta.info()

In [ ]:
mta = mta.iloc[0:2]

In [3]:
mta = pmd.add_datetime(mta)

In [12]:
mta.sample()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,datetime
120738,PTH21,R540,01-00-02,PATH WTC 2,1,PTH,04/14/2018,00:58:21,REGULAR,130524,261315,2018-04-14 00:58:21


In [13]:
mta.loc[120738, 'datetime']

120738   2018-04-23 11:42:01
120738   2018-04-14 00:58:21
120738   2018-04-10 02:32:55
120738   2018-04-02 06:59:21
Name: datetime, dtype: datetime64[ns]